In [ ]:
# Create an environment with devbio-napari installed in it.
mamba create --name neurite-devbio-napari-env python=3.9 devbio-napari -c conda-forge

# Activate the environment
conda activate neurite-devbio-napari-env

# Open Jupyter
jupyter lab

#### Inspect image shape ####

In [1]:
import os
from skimage.io import imread
import napari

# -------------------------------
# Step 0: Pick an image to inspect
# -------------------------------
folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"

# List all TIFF images
files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
print(f"Found {len(files)} images.")

# Pick one image (index 0 for example)
img_path = os.path.join(folder_dir, files[0])
img = imread(img_path)
print(f"Loaded {files[0]} with shape: {img.shape}")

# -------------------------------
print("\nCheck the shape above.")
print("If your image shape is (H, W, C), channel_axis = -1")
print("If your image shape is (C, H, W), channel_axis = 0")
channel_axis = int(input("Enter the channel axis index: "))
print(f"Using channel_axis = {channel_axis} for this dataset.")


Found 5 images.
Loaded Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488.tif with shape: (11, 2, 2036, 2031)

Check the shape above.
If your image shape is (H, W, C), channel_axis = -1
If your image shape is (C, H, W), channel_axis = 0


Enter the channel axis index:  1


Using channel_axis = 1 for this dataset.


#### Save cell and background masks, pixel sizes, background values ####

In [17]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# CONFIG
# ====================================
CHANNEL1 = "channel1"  # formerly ER
CHANNEL2 = "channel2"  # formerly Lys

# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.basename(path), "x_size": x_size, "y_size": y_size, "pixel_area": pixel_area})
    return pd.DataFrame(pixel_data)

# ====================================
# 3. Check for existing masks
# ====================================
def check_existing_masks(folder_dir, file_name):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    if not os.path.exists(mask_dir):
        return None, None

    base = os.path.splitext(file_name)[0]

    # Cell masks
    cell_masks = sorted([
        os.path.join(mask_dir, f) 
        for f in os.listdir(mask_dir) 
        if f.startswith(base + "_cell") and f.endswith(".tif")
    ])
    # Background mask
    bg_mask_path = os.path.join(mask_dir, f"{base}_background.tif")
    if not os.path.exists(bg_mask_path):
        bg_mask_path = None

    return cell_masks, bg_mask_path

# ====================================
# 4. Measure background from existing mask
# ====================================
def measure_background_from_mask(img, bg_mask_path, file_name, bg_csv):
    mask = tifffile.imread(bg_mask_path).astype(bool)
    MIP = img.max(axis=0)

    # MIP metrics
    c1_vals_MIP = MIP[1][mask]
    c2_vals_MIP = MIP[0][mask]
    bg_c1_MIP     = np.mean(c1_vals_MIP)
    bg_c1_MIP_med = np.median(c1_vals_MIP)
    bg_c2_MIP     = np.mean(c2_vals_MIP)
    bg_c2_MIP_med = np.median(c2_vals_MIP)

    # Full Z-stack metrics
    c1_vals_Z = img[:,1][..., mask].flatten()
    c2_vals_Z = img[:,0][..., mask].flatten()
    bg_c1_Z     = np.mean(c1_vals_Z)
    bg_c1_Z_med = np.median(c1_vals_Z)
    bg_c2_Z     = np.mean(c2_vals_Z)
    bg_c2_Z_med = np.median(c2_vals_Z)

    df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                        bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                      columns=["file",
                               f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                               f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                               f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                               f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
    df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
    print(f"Saved background values for {file_name}")

# ====================================
# 5. Draw masks + background (interactive)
# ====================================
def draw_masks_and_background(img, MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)

    # Cell masks
    mask_layer = viewer.add_shapes(name="Cell masks")
    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        file_basename = os.path.splitext(file_name)[0]
        for idx, mask in enumerate(masks_stack, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            tifffile.imwrite(os.path.join(mask_dir, f"{file_basename}_cell{idx}.tif"), mask_to_save)
        print(f"Saved {len(masks_stack)} masks to: {mask_dir}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)

    # Background
    bg_layer = viewer.add_shapes(name="Background")
    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return
        mask_path = os.path.join(mask_dir, f"{os.path.splitext(file_name)[0]}_background.tif")
        tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8)*255)
        print(f"Saved background region for {file_name}")

        # MIP metrics
        mask_bool = mask.astype(bool)
        c1_vals_MIP = MIP[1][mask_bool]
        c2_vals_MIP = MIP[0][mask_bool]
        bg_c1_MIP     = np.mean(c1_vals_MIP)
        bg_c1_MIP_med = np.median(c1_vals_MIP)
        bg_c2_MIP     = np.mean(c2_vals_MIP)
        bg_c2_MIP_med = np.median(c2_vals_MIP)

        # Full Z-stack metrics
        c1_vals_Z = img[:,1][..., mask_bool].flatten()
        c2_vals_Z = img[:,0][..., mask_bool].flatten()
        bg_c1_Z     = np.mean(c1_vals_Z)
        bg_c1_Z_med = np.median(c1_vals_Z)
        bg_c2_Z     = np.mean(c2_vals_Z)
        bg_c2_Z_med = np.median(c2_vals_Z)

        # Save to CSV
        df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                            bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                          columns=["file",
                                   f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                                   f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                                   f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                                   f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved background values for {file_name}")

    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)

    viewer.show(block=True)

# ====================================
# 6. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    
    pixel_csv = os.path.join(folder_dir, "pixel_sizes.csv")
    bg_csv    = os.path.join(folder_dir, "background_values.csv")
    
    # Remove existing CSVs so we start fresh
    if os.path.exists(pixel_csv):
        os.remove(pixel_csv)
    if os.path.exists(bg_csv):
        os.remove(bg_csv)

    # Save pixel sizes
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(pixel_csv, index=False)

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        print(f"\n=== Processing {base} ===")

        # Check existing masks
        cell_masks, bg_mask_path = check_existing_masks(folder_dir, base)

        if bg_mask_path is not None:
            measure_background_from_mask(img, bg_mask_path, base, bg_csv)
            print(f"Measured background from existing mask for {base}")
        else:
            MIP = img.max(axis=0)
            draw_masks_and_background(img, MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)
        print(f"Finished processing {base}")

# ====================================
# 7. Main
# ====================================
if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Saved background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved background values for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13261.vsi - 007 C560, C488 ===


In [14]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# CONFIG
# ====================================
CHANNEL1 = "channel1"  # formerly ER
CHANNEL2 = "channel2"  # formerly Lys

# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.basename(path), "x_size": x_size, "y_size": y_size, "pixel_area": pixel_area})
    return pd.DataFrame(pixel_data)

# ====================================
# 3. Check for existing masks
# ====================================
def check_existing_masks(folder_dir, file_name):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    if not os.path.exists(mask_dir):
        return None, None

    base = os.path.splitext(file_name)[0]

    # Cell masks
    cell_masks = sorted([
        os.path.join(mask_dir, f) 
        for f in os.listdir(mask_dir) 
        if f.startswith(base + "_cell") and f.endswith(".tif")
    ])
    # Background mask
    bg_mask_path = os.path.join(mask_dir, f"{base}_background.tif")
    if not os.path.exists(bg_mask_path):
        bg_mask_path = None

    return cell_masks, bg_mask_path

# ====================================
# 4. Measure background from existing mask
# ====================================
def measure_background_from_mask(img, bg_mask_path, file_name, bg_csv):
    mask = tifffile.imread(bg_mask_path).astype(bool)
    MIP = img.max(axis=0)

    # MIP metrics
    c1_vals_MIP = MIP[1][mask]
    c2_vals_MIP = MIP[0][mask]
    bg_c1_MIP     = np.mean(c1_vals_MIP)
    bg_c1_MIP_med = np.median(c1_vals_MIP)
    bg_c2_MIP     = np.mean(c2_vals_MIP)
    bg_c2_MIP_med = np.median(c2_vals_MIP)

    # Full Z-stack metrics
    c1_vals_Z = img[:,1][..., mask].flatten()
    c2_vals_Z = img[:,0][..., mask].flatten()
    bg_c1_Z     = np.mean(c1_vals_Z)
    bg_c1_Z_med = np.median(c1_vals_Z)
    bg_c2_Z     = np.mean(c2_vals_Z)
    bg_c2_Z_med = np.median(c2_vals_Z)

    df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                        bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                      columns=["file",
                               f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                               f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                               f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                               f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
    df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
    print(f"Saved background values for {file_name}")

# ====================================
# 5. Draw masks + background (interactive)
# ====================================
def draw_masks_and_background(img, MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)

    # Cell masks
    mask_layer = viewer.add_shapes(name="Cell masks")
    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        file_basename = os.path.splitext(file_name)[0]
        for idx, mask in enumerate(masks_stack, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            tifffile.imwrite(os.path.join(mask_dir, f"{file_basename}_cell{idx}.tif"), mask_to_save)
        print(f"Saved {len(masks_stack)} masks to: {mask_dir}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)

    # Background
    bg_layer = viewer.add_shapes(name="Background")
    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return
        mask_path = os.path.join(mask_dir, f"{os.path.splitext(file_name)[0]}_background.tif")
        tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8)*255)
        print(f"Saved background region for {file_name}")

        # MIP metrics
        mask_bool = mask.astype(bool)
        c1_vals_MIP = MIP[1][mask_bool]
        c2_vals_MIP = MIP[0][mask_bool]
        bg_c1_MIP     = np.mean(c1_vals_MIP)
        bg_c1_MIP_med = np.median(c1_vals_MIP)
        bg_c2_MIP     = np.mean(c2_vals_MIP)
        bg_c2_MIP_med = np.median(c2_vals_MIP)

        # Full Z-stack metrics
        c1_vals_Z = img[:,1][..., mask_bool].flatten()
        c2_vals_Z = img[:,0][..., mask_bool].flatten()
        bg_c1_Z     = np.mean(c1_vals_Z)
        bg_c1_Z_med = np.median(c1_vals_Z)
        bg_c2_Z     = np.mean(c2_vals_Z)
        bg_c2_Z_med = np.median(c2_vals_Z)

        # Save to CSV
        df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                            bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                          columns=["file",
                                   f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                                   f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                                   f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                                   f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved background values for {file_name}")

    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)

    viewer.show(block=True)

# ====================================
# 6. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_csv = os.path.join(folder_dir, "pixel_sizes.csv")
    bg_csv    = os.path.join(folder_dir, "background_values.csv")
    
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(pixel_csv, index=False)

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        print(f"\n=== Processing {base} ===")
        
        # Check existing masks
        cell_masks, bg_mask_path = check_existing_masks(folder_dir, base)

        if bg_mask_path is not None:
            measure_background_from_mask(img, bg_mask_path, base, bg_csv)
            print(f"Measured background from existing mask for {base}")
        else:
            MIP = img.max(axis=0)
            draw_masks_and_background(img, MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)
        print(f"Finished processing {base}")

# ====================================
# 7. Main
# ====================================
if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Saved background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved background values for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13261.vsi - 007 C560, C488 ===


In [10]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    import tifffile
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.basename(path), "x_size": x_size, "y_size": y_size, "pixel_area": pixel_area})
    return pd.DataFrame(pixel_data)

# ====================================
# 3. Check for existing masks
# ====================================
def check_existing_masks(folder_dir, file_name):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    if not os.path.exists(mask_dir):
        return None, None

    base = os.path.splitext(file_name)[0]

    # Cell masks
    cell_masks = sorted([
        os.path.join(mask_dir, f) 
        for f in os.listdir(mask_dir) 
        if f.startswith(base + "_cell") and f.endswith(".tif")
    ])
    # Background mask
    bg_mask_path = os.path.join(mask_dir, f"{base}_background.tif")
    if not os.path.exists(bg_mask_path):
        bg_mask_path = None

    return cell_masks, bg_mask_path

# ====================================
# 4. Measure background from existing mask
# ====================================
def measure_background_from_mask(img, bg_mask_path, file_name, bg_csv):
    mask = tifffile.imread(bg_mask_path).astype(bool)

    # Create MIP
    MIP = img.max(axis=0)

    # ===== MIP Background =====
    c1_vals_MIP = MIP[1][mask]
    c2_vals_MIP = MIP[0][mask]

    bg_c1_MIP       = np.mean(c1_vals_MIP)
    bg_c1_MIP_med   = np.median(c1_vals_MIP)
    bg_c2_MIP       = np.mean(c2_vals_MIP)
    bg_c2_MIP_med   = np.median(c2_vals_MIP)

    # ===== Full Z-stack Background =====
    c1_vals_Z = img[:,1][..., mask].flatten()
    c2_vals_Z = img[:,0][..., mask].flatten()

    bg_c1_Z       = np.mean(c1_vals_Z)
    bg_c1_Z_med   = np.median(c1_vals_Z)
    bg_c2_Z       = np.mean(c2_vals_Z)
    bg_c2_Z_med   = np.median(c2_vals_Z)

    import pandas as pd

    df = pd.DataFrame([[
        file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med, 
        bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med
    ]], columns=[
        "file",
        f"Bg_{CHANNEL1}_MIP",
        f"Bg_{CHANNEL1}_MIP_median",
        f"Bg_{CHANNEL1}_Zmean",
        f"Bg_{CHANNEL1}_Zmedian",
        f"Bg_{CHANNEL2}_MIP",
        f"Bg_{CHANNEL2}_MIP_median",
        f"Bg_{CHANNEL2}_Zmean",
        f"Bg_{CHANNEL2}_Zmedian"
    ])
    df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
    print(f"Saved initial background values for {file_name}")


# ====================================
# 3. Draw masks + background on MIP
# ====================================
def draw_masks_and_background(MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    
    os.makedirs(mask_dir, exist_ok=True)
    
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)
    
    # --- Cell masks layer ---
    mask_layer = viewer.add_shapes(name="Cell masks")
    
    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])  # shape (N_cells, H, W)
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        
        MasksList = []
        for m in range(masks_stack.shape[0]):
            Mask = masks_stack[m, :, :].squeeze()
            MasksList.append(Mask)
        
        file_basename = os.path.splitext(os.path.basename(file_name))[0]
        for idx, mask in enumerate(MasksList, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            mask_filename = f"{file_basename}_cell{idx}.tif"
            mask_path = os.path.join(mask_dir, mask_filename)
            tifffile.imwrite(mask_path, mask_to_save)
        
        print(f"Saved {len(MasksList)} masks to: {mask_dir}")
    
    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)
    
    # --- Background layer ---
    bg_layer = viewer.add_shapes(name="Background")
    
    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return
        mask_path = os.path.join(mask_dir, f"{os.path.splitext(file_name)[0]}_background.tif")
        tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8)*255)
        print(f"Saved background region for {file_name}")

        # Save initial background values
        ER_MIP  = np.mean(MIP[1][mask])
        Lys_MIP = np.mean(MIP[0][mask])
        import pandas as pd
        df = pd.DataFrame([[file_name, ER_MIP, Lys_MIP]], columns=["file","Bg_ER_MIP","Bg_Lys_MIP"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved initial background values for {file_name}")
    
    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)
    
    viewer.show(block=True)

# ====================================
# 4. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(os.path.join(folder_dir, "pixel_sizes.csv"), index=False)
    
    bg_csv = os.path.join(folder_dir, "background_values.csv")
    
    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        
        print(f"\n=== Processing {base} ===")
        
        # Check existing masks
        cell_masks, bg_mask_path = check_existing_masks(folder_dir, base)

        if bg_mask_path is not None:
            # Measure background automatically
            measure_background_from_mask(img, bg_mask_path, base, bg_csv)
            print(f"Measured background from existing mask for {base}")
        else:
            # No mask exists, launch interactive viewer
            MIP = np.array(img.max(axis=0))  # (C, H, W)
            draw_masks_and_background(img, MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)

        print(f"Finished processing {base}")


if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    
    # For images (Z, C, H, W): channel_axis=0, after maximum projection in z
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Saved initial background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved initial background values for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13261.vsi - 007

TypeError: draw_masks_and_background() got multiple values for argument 'channel_axis'

In [9]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# CONFIG
# ====================================
CHANNEL1 = "channel1"  # formerly ER
CHANNEL2 = "channel2"  # formerly Lys

# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({
                "file": os.path.basename(path),
                "x_size": x_size,
                "y_size": y_size,
                "pixel_area": pixel_area
            })
    return pd.DataFrame(pixel_data)

# ====================================
# 3. Check for existing masks
# ====================================
def check_existing_masks(folder_dir, file_name):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    if not os.path.exists(mask_dir):
        return None, None

    base = os.path.splitext(file_name)[0]

    # Cell masks
    cell_masks = sorted([
        os.path.join(mask_dir, f) 
        for f in os.listdir(mask_dir) 
        if f.startswith(base + "_cell") and f.endswith(".tif")
    ])
    # Background mask
    bg_mask_path = os.path.join(mask_dir, f"{base}_background.tif")
    if not os.path.exists(bg_mask_path):
        bg_mask_path = None

    return cell_masks, bg_mask_path

# ====================================
# 4. Measure background from existing mask
# ====================================
def measure_background_from_mask(img, bg_mask_path, file_name, bg_csv):
    mask = tifffile.imread(bg_mask_path).astype(bool)

    # Create MIP
    MIP = img.max(axis=0)

    # ===== MIP Background =====
    c1_vals_MIP = MIP[1][mask]
    c2_vals_MIP = MIP[0][mask]

    bg_c1_MIP       = np.mean(c1_vals_MIP)
    bg_c1_MIP_med   = np.median(c1_vals_MIP)
    bg_c2_MIP       = np.mean(c2_vals_MIP)
    bg_c2_MIP_med   = np.median(c2_vals_MIP)

    # ===== Full Z-stack Background =====
    c1_vals_Z = img[:,1][..., mask].flatten()
    c2_vals_Z = img[:,0][..., mask].flatten()

    bg_c1_Z       = np.mean(c1_vals_Z)
    bg_c1_Z_med   = np.median(c1_vals_Z)
    bg_c2_Z       = np.mean(c2_vals_Z)
    bg_c2_Z_med   = np.median(c2_vals_Z)

    import pandas as pd

    df = pd.DataFrame([[
        file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med, 
        bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med
    ]], columns=[
        "file",
        f"Bg_{CHANNEL1}_MIP",
        f"Bg_{CHANNEL1}_MIP_median",
        f"Bg_{CHANNEL1}_Zmean",
        f"Bg_{CHANNEL1}_Zmedian",
        f"Bg_{CHANNEL2}_MIP",
        f"Bg_{CHANNEL2}_MIP_median",
        f"Bg_{CHANNEL2}_Zmean",
        f"Bg_{CHANNEL2}_Zmedian"
    ])
    df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
    print(f"Saved initial background values for {file_name}")

# ====================================
# 5. Draw masks + background (interactive)
# ====================================
def draw_masks_and_background(img, MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)

    # --- Cell masks ---
    mask_layer = viewer.add_shapes(name="Cell masks")

    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        
        file_basename = os.path.splitext(os.path.basename(file_name))[0]
        for idx, mask in enumerate(masks_stack, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            mask_filename = f"{file_basename}_cell{idx}.tif"
            tifffile.imwrite(os.path.join(mask_dir, mask_filename), mask_to_save)

        print(f"Saved {len(masks_stack)} masks to: {mask_dir}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)

    # --- Background layer ---
    bg_layer = viewer.add_shapes(name="Background")

    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return

        # Save background mask
        tifffile.imwrite(
            os.path.join(mask_dir, f"{file_name}_background.tif"),
            (mask > 0).astype(np.uint8)*255
        )
        print(f"Saved background region for {file_name}")

        # Compute background metrics
        mask_bool = mask.astype(bool)

        # MIP metrics
        c1_vals_MIP = MIP[1][mask_bool]
        c2_vals_MIP = MIP[0][mask_bool]
        bg_c1_MIP       = np.mean(c1_vals_MIP)
        bg_c1_MIP_med   = np.median(c1_vals_MIP)
        bg_c2_MIP       = np.mean(c2_vals_MIP)
        bg_c2_MIP_med   = np.median(c2_vals_MIP)

        # Full Z-stack metrics
        c1_vals_Z = img[:,1][..., mask_bool].flatten()
        c2_vals_Z = img[:,0][..., mask_bool].flatten()
        bg_c1_Z       = np.mean(c1_vals_Z)
        bg_c1_Z_med   = np.median(c1_vals_Z)
        bg_c2_Z       = np.mean(c2_vals_Z)
        bg_c2_Z_med   = np.median(c2_vals_Z)

        import pandas as pd
        df = pd.DataFrame([[
            file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med, bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med
        ]], columns=[
            "file", f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median", f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian", f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median", f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"
        ])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved initial background values for {file_name}")
        
    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)
    
    viewer.show(block=True)

# ====================================
# 6. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_csv = os.path.join(folder_dir, "pixel_sizes.csv")
    pixel_df.to_csv(pixel_csv, index=False)
    
    
    bg_csv = os.path.join(folder_dir, "background_values.csv")

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        
        print(f"\n=== Processing {base} ===")

        # Check existing masks
        cell_masks, bg_mask_path = check_existing_masks(folder_dir, base)

        if bg_mask_path is not None:
            # Measure background automatically
            measure_background_from_mask(img, bg_mask_path, base, bg_csv)
            print(f"Measured background from existing mask for {base}")
        else:
            # No mask exists, launch interactive viewer
            MIP = np.array(img.max(axis=0))  # (C, H, W)
            draw_masks_and_background(img, MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)

        print(f"Finished processing {base}")

    # Merge pixel sizes and background CSVs 
    if os.path.exists(pixel_csv) and os.path.exists(bg_csv):
        pixel_df = pd.read_csv(pixel_csv)
        bg_df = pd.read_csv(bg_csv)
        merged_df = pixel_df.merge(bg_df, on="file", how="left")
        output_csv = os.path.join(folder_dir, "image_metadata.csv")
        merged_df.to_csv(output_csv, index=False)
        print(f"\nMerged CSV saved to: {output_csv}")

        # Delete original CSVs
        os.remove(pixel_csv)
        os.remove(bg_csv)
    else:
        print("Pixel or background CSV missing, merge skipped.")

    print("\n=======================================")
    print("Saved final metadata CSV to:")
    print(output_csv)
    print("=======================================")

# ====================================
# 7. Main
# ====================================
if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Saved initial background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved initial background values for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Measured background from existing mask for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13261.vsi - 007

In [2]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# CONFIG
# ====================================
CHANNEL1 = "channel1"  # formerly ER
CHANNEL2 = "channel2"  # formerly Lys


# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames


# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({
                "file": os.path.basename(path),
                "x_size": x_size,
                "y_size": y_size,
                "pixel_area": pixel_area
            })
    return pd.DataFrame(pixel_data)


# ====================================
# 3. Draw masks + background
#    Now returns metadata instead of writing CSV
# ====================================
def draw_masks_and_background(img, MIP, file_name, mask_dir, channel_axis=0):
    
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)

    # --- Cell masks ---
    mask_layer = viewer.add_shapes(name="Cell masks")

    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        
        file_basename = os.path.splitext(os.path.basename(file_name))[0]
        for idx, mask in enumerate(masks_stack, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            mask_filename = f"{file_basename}_cell{idx}.tif"
            tifffile.imwrite(os.path.join(mask_dir, mask_filename), mask_to_save)

        print(f"Saved {len(masks_stack)} masks to: {mask_dir}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)


    # --- Background layer ---
    bg_layer = viewer.add_shapes(name="Background")

    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return

        # Save background mask image
        tifffile.imwrite(
            os.path.join(mask_dir, f"{file_name}_background.tif"),
            (mask > 0).astype(np.uint8)*255
        )
        print(f"Saved background region for {file_name}")

        # --- Compute background values ---
        mask_bool = mask.astype(bool)

        # ===== MIP Background =====
        c1_vals_MIP = MIP[1][mask_bool]
        c2_vals_MIP = MIP[0][mask_bool]

        bg_c1_MIP       = np.mean(c1_vals_MIP)
        bg_c1_MIP_med   = np.median(c1_vals_MIP)
        bg_c2_MIP       = np.mean(c2_vals_MIP)
        bg_c2_MIP_med   = np.median(c2_vals_MIP)

        # ===== Full Z-stack Background =====
        c1_vals_Z = img[:,1][..., mask_bool].flatten()
        c2_vals_Z = img[:,0][..., mask_bool].flatten()

        bg_c1_Z       = np.mean(c1_vals_Z)
        bg_c1_Z_med   = np.median(c1_vals_Z)
        bg_c2_Z       = np.mean(c2_vals_Z)
        bg_c2_Z_med   = np.median(c2_vals_Z)

        print(f"Recorded background values for {file_name}")

        return {
            "file": file_name,
            f"Bg_{CHANNEL1}_MIP": bg_c1_MIP,
            f"Bg_{CHANNEL1}_MIP_median": bg_c1_MIP_med,
            f"Bg_{CHANNEL1}_Zmean": bg_c1_Z,
            f"Bg_{CHANNEL1}_Zmedian": bg_c1_Z_med,

            f"Bg_{CHANNEL2}_MIP": bg_c2_MIP,
            f"Bg_{CHANNEL2}_MIP_median": bg_c2_MIP_med,
            f"Bg_{CHANNEL2}_Zmean": bg_c2_Z,
            f"Bg_{CHANNEL2}_Zmedian": bg_c2_Z_med,
        }

    # This captures saved data and makes it available after viewer closes
    result_holder = {"value": None}

    def wrapped_bg_save():
        result = save_background()
        if result is not None:
            result_holder["value"] = result

    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(wrapped_bg_save)
    viewer.window.add_dock_widget(button_bg)

    viewer.show(block=True)
    return result_holder["value"]


# ====================================
# 4. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    
    bg_records = []

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        
        print(f"\n=== Processing {base} ===")

        MIP = np.array(img.max(axis=0))  # (C, H, W)
        bg_values = draw_masks_and_background(img, MIP, base, mask_dir, channel_axis=channel_axis)

        if bg_values is not None:
            bg_records.append(bg_values)

        print(f"Finished processing {base}")

    if len(bg_records) == 0:
        print("\nNo background data saved! CSV will contain only pixel sizes.")
        merged = pixel_df
    else:
        bg_df = pd.DataFrame(bg_records)
        merged = pixel_df.merge(bg_df, on="file", how="left")

    output_csv = os.path.join(folder_dir, "image_metadata.csv")
    merged.to_csv(output_csv, index=False)

    print("\n=======================================")
    print("Saved final metadata CSV to:")
    print(output_csv)
    print("=======================================")


if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Assistant skips harvesting pyclesperanto as it's not installed.
Saved 1 masks to: C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run\CellMasks
Saved background region for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Saved background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved 2 masks to: C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run\CellMasks
Saved background region for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Saved background values for Experiment (3)_Mu

In [2]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

# ====================================
# 1. Load images
# ====================================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ====================================
# 2. Extract pixel sizes
# ====================================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    import tifffile
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.basename(path), "x_size": x_size, "y_size": y_size, "pixel_area": pixel_area})
    return pd.DataFrame(pixel_data)

# ====================================
# 3. Draw masks + background on MIP
# ====================================
def draw_masks_and_background(MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    
    os.makedirs(mask_dir, exist_ok=True)
    
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)
    
    # --- Cell masks layer ---
    mask_layer = viewer.add_shapes(name="Cell masks")
    
    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])  # shape (N_cells, H, W)
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        
        MasksList = []
        for m in range(masks_stack.shape[0]):
            Mask = masks_stack[m, :, :].squeeze()
            MasksList.append(Mask)
        
        file_basename = os.path.splitext(os.path.basename(file_name))[0]
        for idx, mask in enumerate(MasksList, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            mask_filename = f"{file_basename}_cell{idx}.tif"
            mask_path = os.path.join(mask_dir, mask_filename)
            tifffile.imwrite(mask_path, mask_to_save)
        
        print(f"Saved {len(MasksList)} masks to: {mask_dir}")
    
    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)
    
    # --- Background layer ---
    bg_layer = viewer.add_shapes(name="Background")
    
    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return
        mask_path = os.path.join(mask_dir, f"{os.path.splitext(file_name)[0]}_background.tif")
        tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8)*255)
        print(f"Saved background region for {file_name}")

        # Save initial background values
        ER_MIP  = np.mean(MIP[1][mask])
        Lys_MIP = np.mean(MIP[0][mask])
        import pandas as pd
        df = pd.DataFrame([[file_name, ER_MIP, Lys_MIP]], columns=["file","Bg_ER_MIP","Bg_Lys_MIP"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved initial background values for {file_name}")
    
    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)
    
    viewer.show(block=True)

# ====================================
# 4. Run pipeline
# ====================================
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(os.path.join(folder_dir, "pixel_sizes.csv"), index=False)
    
    bg_csv = os.path.join(folder_dir, "background_values.csv")
    
    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        
        print(f"\n=== Processing {base} ===")
        
        MIP = np.array(img.max(axis=0))  # (C, H, W)
        draw_masks_and_background(MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)
        
        print(f"Finished processing {base}")


if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
    
    # For images (Z, C, H, W): channel_axis=0, after maximum projection in z
    run_pipeline(folder_dir, channel_axis=0)


Loaded 5 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run

=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===
Assistant skips harvesting pyclesperanto as it's not installed.
Saved 1 masks to: C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run\CellMasks
Saved background region for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Saved initial background values for Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488
Finished processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===
Saved 2 masks to: C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run\CellMasks
Saved background region for Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488
Saved initial background values for E

#### Open each cell, save a masked cell within the bounding box, also save the cell area ####

In [3]:
import os
import numpy as np
import tifffile
import napari
import pandas as pd
from qtpy.QtWidgets import QPushButton
from qtpy.QtWidgets import QApplication
import threading
import time


# =========================================================
# 1. Extract cropped 4D cell stack + mask area + bbox area
# =========================================================
def extract_cell_stack(img_4d, mask_path):
    mask = tifffile.imread(mask_path) > 0

    rows, cols = np.nonzero(mask)
    if len(rows) == 0:
        return None, None, None, None

    min_r, max_r = np.min(rows), np.max(rows)
    min_c, max_c = np.min(cols), np.max(cols)

    cropped = img_4d[:, :, min_r:max_r+1, min_c:max_c+1]

    mask_area = np.sum(mask)
    bbox_area = (max_r - min_r + 1) * (max_c - min_c + 1)

    return cropped, mask_area, bbox_area, (min_r, max_r, min_c, max_c)


# =========================================================
# 2. Save cropped cell stack to TIFF
# =========================================================
def save_cropped_cell(cropped_stack, output_dir, base, cell_index):
    os.makedirs(output_dir, exist_ok=True)

    save_path = os.path.join(output_dir, f"{base}_cell{cell_index}.tif")
    tifffile.imwrite(save_path, cropped_stack.astype(cropped_stack.dtype), imagej=True, metadata={'axes': 'ZCYX'})

    return save_path


# ====================================
# 3. Run pipeline
# ====================================
def process_cells(folder_dir):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    output_dir = os.path.join(folder_dir, "Cells")
    os.makedirs(output_dir, exist_ok=True)

    csv_path = os.path.join(folder_dir, "cell_measurements.csv")
    cells_list = []
    cell_names = []
    entries = []

    # Loop over original images
    for fname in os.listdir(folder_dir):
        if not fname.endswith(".tif"):
            continue

        base = os.path.splitext(fname)[0]
        print(f"\n=== Processing {base} ===")

        img_path = os.path.join(folder_dir, fname)
        img_4d = tifffile.imread(img_path)

        # Collect masks for this image
        file_basename = os.path.splitext(os.path.basename(base))[0]
        mask_files = sorted(
            [f for f in os.listdir(mask_dir) if f.startswith(file_basename + "_cell")]
        )

        if len(mask_files) == 0:
            print("No masks found for this image.")
            continue

        for idx, mask_file in enumerate(mask_files, start=1):
            mask_path = os.path.join(mask_dir, mask_file)

            # --- 1. Extract cropped stack ---
            cropped, mask_area, bbox_area, bbox_coords = extract_cell_stack(img_4d, mask_path)

            if cropped is None:
                print(f"Empty mask for {mask_file}")
                continue

            # --- 2. Save cropped TIFF ---
            cell_tif_path = save_cropped_cell(cropped, output_dir, file_basename, idx)


            # --- Collect results ---
            entries.append([
                file_basename,
                idx,
                mask_area,
                bbox_area,
                cell_tif_path
            ])

    # Save results to CSV
    df = pd.DataFrame(
        entries,
        columns=["image", "cell_index", "mask_area", "bbox_area", "cell_tif_path"]
    )
    df.to_csv(csv_path, index=False)

    print(f"\nSaved measurements to: {csv_path}")

def run_cell_pipeline(folder_dir):
    """
    Run AFTER mask drawing pipeline.
    Extract each cell, save TIFF, compute areas
    """
    process_cells(folder_dir)


folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run"
run_cell_pipeline(folder_dir)


=== Processing Experiment (3)_Multichannel tZ-Stack_20250515b_13239.vsi - 001 C560, C488 ===

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13257.vsi - 003 C560, C488 ===

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13261.vsi - 007 C560, C488 ===

=== Processing Experiment_Multichannel tZ-Stack_20250515a_13203.vsi - 003 C560, C488 ===

=== Processing Experiment_Multichannel tZ-Stack_20250515a_13206.vsi - 006 C560, C488 ===

Saved measurements to: C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\FA MAPPER test run\cell_measurements.csv


#### Manually open each cell in imageJ and make a csv file with the filename and focal plane ####

#### Open each image and segment the focal adhesions and MAPPER puncta ####

#### Old code that might have a use ####

In [3]:
import os
import numpy as np
import tifffile
import pandas as pd
from skimage.io import imread
import napari
from PyQt5.QtWidgets import QPushButton

# ==========================
# 1. Load TIFF images
# ==========================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ==========================
# 2. Pixel size extraction
# ==========================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1 / x_res if x_res else None
            y_size = 1 / y_res if y_res else None
            area = x_size * y_size if x_size and y_size else None
            pixel_data.append({
                "file": os.path.basename(path),
                "x_size": x_size, "y_size": y_size, "area": area
            })
    return pd.DataFrame(pixel_data)

# ==========================
# 3. Draw cell masks + background
# ==========================
def draw_masks_and_background(img, file_name, mask_dir, bg_csv):
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(img, name=file_name)

    # ---- Cell masks ----
    mask_layer = viewer.add_shapes(name="Cell masks")

    def save_masks():
        masks = mask_layer.to_masks(img.shape[:2])
        for i, mask in enumerate(masks):
            mask_path = os.path.join(mask_dir, f"{file_name}_cell{i+1}.tif")
            tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8) * 255)
        print(f"Saved {len(masks)} cell masks for {file_name}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks)
    viewer.window.add_dock_widget(button_masks)

    # ---- Background ----
    bg_layer = viewer.add_shapes(name="Background")

    def save_background():
        mask = bg_layer.to_masks(img.shape[:2]).squeeze()
        if mask.sum() == 0:
            print("No background region drawn!")
            return
        ERmean = np.mean(img[:, :, 1][mask > 0])
        Lysmean = np.mean(img[:, :, 0][mask > 0])
        df = pd.DataFrame([[file_name, ERmean, Lysmean]],
                          columns=["file", "ER_CN", "Lys_CN"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved background values for {file_name}")

    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)

    # ---- Block until window closed ----
    viewer.show(block=True)
    napari.run()

# ==========================
# 4. Draw ER sheets per cell
# ==========================
def draw_ersheets_per_cell(img, file_name, mask_path, output_dir, cell_index):
    m = imread(mask_path)
    rows, cols = np.nonzero(m)
    min_r, max_r = np.min(rows), np.max(rows)
    min_c, max_c = np.min(cols), np.max(cols)

    cell_crop = img[min_r:max_r+1, min_c:max_c+1, :]
    mask_crop = m[min_r:max_r+1, min_c:max_c+1]

    viewer = napari.view_image(cell_crop, name=f"{file_name}_cell{cell_index}")
    shapes_layer = viewer.add_shapes(name="ER sheets")

    def save_sheets():
        mask = shapes_layer.to_masks(cell_crop.shape[:2]).max(axis=0)
        mask = mask & (mask_crop > 0)
        out_path = os.path.join(output_dir, f"{file_name}_cell{cell_index}_sheets.tif")
        tifffile.imwrite(out_path, (mask > 0).astype(np.uint8) * 255)
        print(f"Saved ER sheets mask for cell {cell_index}")

    button = QPushButton("Save ER sheets mask")
    button.clicked.connect(save_sheets)
    viewer.window.add_dock_widget(button)
    
    viewer.show(block=True)
    napari.run()  # wait until user closes

# ==========================
# 5. Main pipeline
# ==========================
def run_pipeline(folder_dir):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(os.path.join(folder_dir, "pixel_sizes.csv"), index=False)

    bg_csv = os.path.join(folder_dir, "background_values.csv")

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")

        print(f"\n=== Processing {base} ===")

        # Step 1 + 2: Draw masks + background (single viewer)
        draw_masks_and_background(img, base, mask_dir, bg_csv)

        # Step 3: For each saved cell mask, draw ER sheets in separate viewers
        mask_files = [f for f in os.listdir(mask_dir) if f.startswith(base + "_cell")]
        mask_files.sort()
        for idx, f in enumerate(mask_files, start=1):
            draw_ersheets_per_cell(img, base, os.path.join(mask_dir, f), mask_dir, idx)

# ==========================
# Run the pipeline
# ==========================
if __name__ == "__main__":
    folder_dir = r"C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\First frames of FA and MAPPER movies with DMSO\DMSO_Glass"
    run_pipeline(folder_dir)



Loaded 7 images from C:\Users\jonatmt\OneDrive - Universitetet i Oslo\Desktop\First frames of FA and MAPPER movies with DMSO\DMSO_Glass

=== Processing Experiment (3)_Multichannel tZ-Stack_20250516_13255.vsi - 001 C560, C488 ===
No background region drawn!


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
import os
import numpy as np
import tifffile
import warnings
import pandas as pd
from skimage.io import imread
import napari
from PyQt5.QtWidgets import QPushButton

# ==========================
# 1. Load TIFF images
# ==========================
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# ==========================
# 2. Pixel size extraction
# ==========================
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.basename(path), "x_size": x_size, "y_size": y_size, "area": area})
    return pd.DataFrame(pixel_data)

# ==========================
# 3. Draw cell masks
# ==========================
def draw_cell_masks(image, file_name, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    viewer = napari.view_image(image)
    shapes_layer = viewer.add_shapes()

    def save_masks():
        masks = shapes_layer.to_masks(image.shape[:2])
        for i, mask in enumerate(masks):
            mask_path = os.path.join(output_dir, f"{file_name}_cell{i+1}.tif")
            tifffile.imwrite(mask_path, (mask>0).astype(np.uint8)*255)
        print(f"Saved {len(masks)} cell masks for {file_name}")

    button = QPushButton('Save cell masks')
    button.clicked.connect(save_masks)
    viewer.window.add_dock_widget(button)
    napari.run()

# ==========================
# 4. Draw background region
# ==========================
def draw_background_region(image, file_name, output_csv):
    viewer = napari.view_image(image)
    shapes_layer = viewer.add_shapes()

    def save_background():
        mask = shapes_layer.to_masks(image.shape[:2]).squeeze()
        ERmean = np.mean(image[:,:,1][mask>0])
        Lysmean = np.mean(image[:,:,0][mask>0])
        df = pd.DataFrame([[file_name, ERmean, Lysmean]], columns=["file","ER_CN","Lys_CN"])
        df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)
        print(f"Saved background values for {file_name}")

    button = QPushButton('Save background region')
    button.clicked.connect(save_background)
    viewer.window.add_dock_widget(button)
    napari.run()

# ==========================
# 5. Draw ER sheets for each cell
# ==========================
def draw_ERsheets(CellsList, MasksListBB, file_name, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    ERsheetsList = []
    for i, cell in enumerate(CellsList):
        viewer = napari.view_image(cell, rgb=False, channel_axis=-1, colormap=['magenta','green','blue'])
        shapes_layer = viewer.add_shapes()

        def save_sheets():
            mask = shapes_layer.to_masks(cell.shape[:2]).max(axis=0)
            mask = mask & (MasksListBB[i]>0)
            mask_path = os.path.join(output_dir, f"{file_name}_cell{i+1}_sheets.tif")
            tifffile.imwrite(mask_path, (mask>0).astype(np.uint8)*255)
            print(f"Saved ER sheets mask for cell {i+1}")

        button = QPushButton('Save ER sheets mask')
        button.clicked.connect(save_sheets)
        viewer.window.add_dock_widget(button)
        napari.run()
        ERsheetsList.append(mask)
    return ERsheetsList

# ==========================
# 6. Main pipeline
# ==========================
def run_pipeline(folder_dir):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(os.path.join(folder_dir, "pixel_sizes.csv"), index=False)

    bg_csv = os.path.join(folder_dir, "background_values.csv")

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")

        # Step 1: Cell masks
        draw_cell_masks(img, base, mask_dir)

        # Step 2: Background region
        MaxImCombine = np.amax(img, axis=2)
        draw_background_region(MaxImCombine, base, bg_csv)

        # Step 3: Apply cell masks for ER sheets
        MasksList = [imread(os.path.join(mask_dir,f)) for f in os.listdir(mask_dir) if f.startswith(base+"_cell")]
        CellsList, MasksListBB = [], []
        for m in MasksList:
            rows, cols = np.nonzero(m)
            min_r, max_r = np.min(rows), np.max(rows)
            min_c, max_c = np.min(cols), np.max(cols)
            MasksListBB.append(m[min_r:max_r+1, min_c:max_c+1])
            CellsList.append(img[min_r:max_r+1, min_c:max_c+1, :])

        # Step 4: ER sheets
        draw_ERsheets(CellsList, MasksListBB, base, mask_dir)

# ==========================
# Run the pipeline
# ==========================
if __name__ == "__main__":
    folder_dir = "C:/Users/jonatmt\OneDrive - Universitetet i Oslo/Desktop/First frames of FA and MAPPER movies with DMSO/DMSO_Glass"
    run_pipeline(folder_dir)


Loaded 7 images from C:/Users/jonatmt\OneDrive - Universitetet i Oslo/Desktop/First frames of FA and MAPPER movies with DMSO/DMSO_Glass
Assistant skips harvesting pyclesperanto as it's not installed.


KeyboardInterrupt: 

In [16]:
import os
import numpy as np
import pandas as pd
import tifffile
from skimage.io import imread
from skimage.morphology import h_maxima, remove_small_objects, disk, white_tophat
from skimage.filters import gaussian, threshold_otsu, laplace, median, threshold_local
from skimage import measure
from skimage.measure import label, regionprops
from skimage.feature import peak_local_max
from skimage.segmentation import watershed, find_boundaries
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import spearmanr, kendalltau
import napari_segment_blobs_and_things_with_membranes as nsbatwm
import pyclesperanto_prototype as cle
from matplotlib.backends.backend_pdf import PdfPages
from skimage import morphology
from scipy.ndimage import distance_transform_edt
import textwrap
from scipy import ndimage as ndi

## Functions

def load_cell_masks(mask_dir, base_name):
    MasksList = []
    for f in os.listdir(mask_dir):
        if f.startswith(base_name + '_cell') and f.endswith('.tif') and '_sheets' not in f:
            MasksList.append(imread(os.path.join(mask_dir, f))>0)
    return MasksList


def load_ERsheets_masks(mask_dir, base_name, num_cells):
    ERsheetsList = []
    for i in range(1, num_cells+1):
        f = os.path.join(mask_dir, f"{base_name}_cell{i}_sheets.tif")
        ERsheetsList.append(imread(f)>0)
    return ERsheetsList

def pearson_correlation_coefficient(image1, image2):
    return np.corrcoef(image1.ravel(), image2.ravel())[0, 1]

def spearman_correlation_coefficient(image1, image2):
    return spearmanr(image1.ravel(), image2.ravel()).correlation

def kendall_tau(image1, image2):
    return kendalltau(image1.ravel(), image2.ravel()).correlation

def manders_coefficients(image1, image2):
    image1 = image1.astype(np.float64)
    image2 = image2.astype(np.float64)
    M1 = np.sum(image1[image2 > 0]) / np.sum(image1) if np.sum(image1) > 0 else 0.0
    M2 = np.sum(image2[image1 > 0]) / np.sum(image2) if np.sum(image2) > 0 else 0.0
    return M1, M2

def overlap_coefficient(image1, image2):
    image1 = image1.astype(np.float64)
    image2 = image2.astype(np.float64)
    return np.sum(np.minimum(image1, image2)) / np.sqrt(np.sum(image1) * np.sum(image2))

def jaccard_index(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union != 0 else 0.0

def dice_coefficient(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    denom = mask1.sum() + mask2.sum()
    return (2.0 * intersection / denom) if denom != 0 else 0.0

def compute_all_coefficients(img1, img2, mask1, mask2, region='All', ch1=1, ch2=2):
    # Intensity-based metrics
    pearson = pearson_correlation_coefficient(img1, img2)
    spearman = spearman_correlation_coefficient(img1, img2)
    kendall = kendall_tau(img1, img2)
    manders_int = manders_coefficients(img1, img2)
    overlap = overlap_coefficient(img1, img2)

    # Mask-based metrics
    jaccard = jaccard_index(mask1, mask2)
    dice = dice_coefficient(mask1, mask2)
    manders_mask = manders_coefficients(mask1, mask2)

    return {
        'Region': region,
        'Channel 1': ch1,
        'Channel 2': ch2,
        # Intensity-based
        'Pearson': pearson,
        'Spearman': spearman,
        'Kendall Tau': kendall,
        'Manders Ch1 with Ch2 (Intensity)': manders_int[0],
        'Manders Ch2 with Ch1 (Intensity)': manders_int[1],
        'Overlap Coefficient': overlap,
        # Mask-based
        'Dice Coefficient': dice,
        'Jaccard Index': jaccard,
        'Manders Ch1 with Ch2 (Mask)': manders_mask[0],
        'Manders Ch2 with Ch1 (Mask)': manders_mask[1],
    }

## Automated pipeline

def automated_analysis(folder_dir):
    # Load lists of images
    ImNames = [f for f in os.listdir(folder_dir) if f.endswith('.tif')]
    ImsFP = [os.path.join(folder_dir, f) for f in ImNames]

    # Load pixel sizes and background
    #pixel_sizes = pd.read_csv(os.path.join(folder_dir, 'pixel_sizes.csv'), index_col=False)
    #background = pd.read_csv(os.path.join(folder_dir, 'background_values.csv'), index_col=False)

    for CurrentIm, (ImPath, file_name) in enumerate(zip(ImsFP, ImNames)):
        base = os.path.splitext(file_name)[0]
        print(f"Processing image {CurrentIm+1}/{len(ImsFP)}: {file_name}")

        Im = imread(ImPath)
        mask_dir = os.path.join(folder_dir, 'CellMasks')

        # Load masks
        MasksList = load_cell_masks(mask_dir, base)
        CellsList, MasksListBB = [], []
        for m in MasksList:
            rows, cols = np.nonzero(m)
            min_r, max_r = np.min(rows), np.max(rows)
            min_c, max_c = np.min(cols), np.max(cols)
            MasksListBB.append(m[min_r:max_r+1, min_c:max_c+1])
            CellsList.append(Im[min_r:max_r+1, min_c:max_c+1, :])

        ERsheetsList = load_ERsheets_masks(mask_dir, base, len(CellsList))

        # Pixel and background
        #pixel_info = pixel_sizes[pixel_sizes.file==file_name].iloc[0]
        #x_pixel_size = pixel_info.x_size
        #y_pixel_size = pixel_info.y_size
        #xy_pixel_area = pixel_info.area
        #bg_info = background[background.file==file_name].iloc[0]
        #ER_CN = bg_info.ER_CN
        #Lys_CN = bg_info.Lys_CN
        ER_CN = 104
        Lys_CN = 104

        for CurrentCell, substack in enumerate(CellsList):
            print(f"  Analyzing cell {CurrentCell+1}/{len(CellsList)}")

            # --- Analysis steps ---
            # Extract channels
            ER = substack[:,:,1]
            Lys = substack[:,:,0]
            Rab18 = substack[:,:,2]
            #CellMask = MasksListBB[CurrentCell]
            #CellMask = CellMask.astype(int)
            CellMask = np.clip(MasksListBB[CurrentCell], 0, 1)
            #ERsheets = ERsheetsList[CurrentCell]
            #ERsheets = ERsheets.astype(int)
            ERsheets = np.clip(ERsheetsList[CurrentCell], 0, 1)
            CellMask_bool = CellMask > 0
            ERsheets_bool = ERsheets > 0

            # ER background subtraction and threshold
            ObjectSize = 5
            ER_sub = np.copy(ER)
            for i in range(ER_sub.shape[0]):
                for j in range(ER_sub.shape[1]):
                    pixel_value = ER_sub[i, j]
                    new_value = max(int(pixel_value - (0.9 * ER_CN)), 0)  # prevent negative
                    ER_sub[i, j] = new_value
            # Mask out regions outside the cell (after filtering!)
            ER_masked = np.where(CellMask == 1, ER_sub, 0)

            # Apply Otsu thresholding to the masked filtered image
            threshold_val = threshold_otsu(ER_masked)
            ERThresh = ER_masked > threshold_val
            
            ERThresh_clean = remove_small_objects(ERThresh, min_size=ObjectSize)

            # Lysosome processing

            # --- Preprocessing ---
            # Subtract camera/ER background
            Lys_sub = np.maximum(Lys - 0.9 * Lys_CN, 0)

            # Median filter to reduce salt-and-pepper noise
            Lys_denoised = median(Lys_sub, disk(1))  # adjust radius if needed

            # Optional top-hat for small bright lysosomes
            Lys_enhanced = Lys_denoised - ndi.grey_opening(Lys_denoised, size=(15,15))
            Lys_enhanced = np.clip(Lys_enhanced, 0, None)

            # Normalize 0-1
            Lys_norm = (Lys_enhanced - Lys_enhanced.min()) / (Lys_enhanced.max() - Lys_enhanced.min() + 1e-9)

            # --- Adaptive thresholding ---
            block_size = 25  # local window size, adjust as needed
            local_thresh = threshold_local(Lys_norm, block_size, offset=-0.05)  # slightly lower to include dim lysosomes
            Lys_thresh = (Lys_norm > local_thresh) * CellMask

            # --- Remove tiny noise ---
            MinBlobSize = 5
            Lys_thresh = (Lys_norm > local_thresh) & (CellMask > 0)  # boolean mask
            Lys_thresh = remove_small_objects(Lys_thresh, min_size=5)

            # --- Optional watershed for clusters ---
            distance = ndi.distance_transform_edt(Lys_thresh)
            coords = peak_local_max(distance, labels=Lys_thresh, min_distance=1)  # small distance to detect small peaks
            local_maxi = np.zeros_like(distance, dtype=bool)
            if coords.size > 0:
                local_maxi[tuple(coords.T)] = True
            markers = label(local_maxi)[0]
            if np.max(markers) > 0:
                Lys_watershed = watershed(-distance, markers, mask=Lys_thresh)
            else:
                Lys_watershed = Lys_thresh.astype(np.uint16)

            # --- Size filtering ---
            props = regionprops(Lys_watershed)
            Lys_final = np.zeros_like(Lys_watershed, dtype=np.uint16)
            if len(props) > 0:
                areas = [p.area for p in props]
                cutoff = max(5, np.percentile(areas, 20))  # remove smallest 20%
                for p in props:
                    area = p.area
                    perimeter = max(p.perimeter, 1)
                    if area >= cutoff:
                        Lys_final[Lys_watershed == p.label] = p.label
                        
            Lys_final = (Lys_final > 0).astype(np.uint8)


            ## Prepare PDF with multiple pages
            # Create a PdfPages object to save multiple figures into a single PDF file
            save_file_name = f"{file_name}_Cell{CurrentCell + 1}"
            pdf_path = os.path.join(folder_dir, f"{save_file_name}.pdf")
            pdf_pages = PdfPages(pdf_path)

            # Page 1: Summary
            fig, axs = plt.subplots(ncols=2, nrows=4, figsize=(10,20))
            axs = axs.ravel()
            for ax, data, title in zip(axs, [CellMask, ERsheets, Lys, Lys_final, ER, ERThresh_clean, Rab18],
                                       ['Cell Mask','ER sheets mask','Original Lys','Thresholded Lys','Original ER','Thresholded ER','Rab18']):
                ax.imshow(data,cmap='gray')
                ax.set_title(title)
                ax.axis('off')
            #fig.text(0.5, 0.01, f"Pixel sizes x={x_pixel_size}um, y={y_pixel_size}um, area={xy_pixel_area}um^2", ha='center')
            pdf_pages.savefig(fig)
            plt.close(fig)

            # Page 2: Lysosome processing
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            ax[0].imshow(Lys, cmap='gray')
            ax[0].set_title("Original Lysosome Channel")
            ax[0].axis('off')

            boundaries = find_boundaries(Lys_final)
            overlay = np.dstack([Lys]*3)
            overlay[boundaries, 0] = 255  # red boundaries
            ax[1].imshow(overlay.astype(np.uint8))
            ax[1].set_title(f"LoG + Watershed + MinSize={int(MinBlobSize)} px")
            ax[1].axis('off')

            plt.tight_layout()
            pdf_pages.savefig(fig)
            plt.close(fig)

            # Page 3: ER processing
            fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(15,10))
            fig.suptitle(textwrap.fill('ER channel', width=40), fontsize=16, fontweight='bold')
            ER_imgs = [ER, ER_sub, ERThresh, ERThresh_clean]
            ER_titles = [
                'Original ER image',
                f'Background subtracted ER (value = {ER_CN})',
                'Otsu Thresholded ER',
                f'Otsu Thresholded and small object cleaned ER (size = 5 px)'
            ]
            for i, (data, title) in enumerate(zip(ER_imgs, ER_titles)):
                ax = axs[i//2, i%2]
                ax.imshow(data, cmap='gray')
                ax.set_title(title)
                ax.axis('off')
            pdf_pages.savefig(fig)
            plt.close(fig)

            pdf_pages.close()

            ## --- Region properties & Co-localisation ---
            Lys_Ana = nsbatwm.connected_component_labeling(Lys_final, False)
            Lys_props = measure.regionprops(Lys_Ana, intensity_image=Lys)
            
            def get_region_property_values(region, property_names):
                property_values = {}
                for prop_name in property_names:
                    prop_value = getattr(region, prop_name, None)
                    if ' ' not in str(prop_value):
                        property_values[prop_name] = prop_value
                return property_values

            # Only if there are any regions
            if len(Lys_props) > 0:
                Lys_property_names = [attr for attr in dir(Lys_props[0]) if not attr.startswith('_')]
                Lys_region_data = [get_region_property_values(region, Lys_property_names) for region in Lys_props]
            else:
                Lys_region_data = []

            Lys_region_data_augmented = []

            for i, region in enumerate(Lys_props):
                region_mask = Lys_Ana == region.label
                intersection = np.logical_and(region_mask, ERThresh_clean)
                intersection_area = intersection.sum()
                intersection_percent = (intersection_area / region.area * 100) if region.area > 0 else 0
                
                # Count connected components of the intersection
                _, num_intersections = measure.label(intersection, return_num=True)
                
                ersheets_intersection_area = np.logical_and(region_mask, ERsheets).sum()
                ersheets_percentage = (ersheets_intersection_area / region.area * 100) if region.area > 0 else 0

                # Take the existing region dictionary and add new metrics
                d = Lys_region_data[i].copy()
                d.update({
                    'Cell area': CellMask_bool.sum(),
                    'ERsheets area': ERsheets_bool.sum(),
                    'ERThresh_intersection_percentage': intersection_percent,
                    'ERThresh_intersection_area': intersection_area,
                    'ERThresh_num_intersection_areas': num_intersections,
                    'ERsheets_intersection_percentage': ersheets_percentage
                })

                Lys_region_data_augmented.append(d)

            # Convert to DataFrame
            Lys_df = pd.DataFrame(Lys_region_data_augmented)
            Lys_df.to_csv(os.path.join(folder_dir, f"{base}_Cell_{CurrentCell+1}_Lysosomes.csv"), index=False)

            # --- Co-localisation metrics ---
            NotSheets_Mask = CellMask - np.multiply(CellMask, ERsheets)

            # Extract subregions for each
            results = []

            # IN SHEETS
            img1_in = Lys * ERsheets
            img2_in = ER * ERsheets
            mask1_in = Lys_final * ERsheets
            mask2_in = ERThresh_clean * ERsheets

            result_in = compute_all_coefficients(img1_in, img2_in, mask1_in, mask2_in, region='In Sheets', ch1='Lys', ch2='ER')
            results.append(result_in)

            # NOT IN SHEETS
            img1_out = Lys * NotSheets_Mask
            img2_out = ER * NotSheets_Mask
            mask1_out = Lys_final * NotSheets_Mask
            mask2_out = ERThresh_clean * NotSheets_Mask

            result_out = compute_all_coefficients(img1_out, img2_out, mask1_out, mask2_out, region='Not In Sheets', ch1='Lys', ch2='ER')
            results.append(result_out)

            # WHOLE CELL
            Lys_Cell = Lys * CellMask
            ER_Cell = ER * CellMask
            
            result_whole = compute_all_coefficients(
                Lys_Cell, ER_Cell,
                Lys_final, ERThresh_clean,
                region='Whole Cell', 
                ch1='Lys', 
                ch2='ER'
            )
            results.append(result_whole)

            df = pd.DataFrame(results)

            # -----------------------------
            # Save results
            # -----------------------------
            pd.DataFrame(results).to_csv( os.path.join(folder_dir, f"{base}_Cell_{CurrentCell+1}_CoLocalisation.csv"), index=False )

            
## Run the code

if __name__ == '__main__':
    #folder_dir = 'C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Desktop/TestRun'
    #folder_dir = 'C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Documents/Progida Postdoc/Image data/ER LYS images from Amalie/TIFF files/20240307/TIFFs' #Lysosomes in channel 3
    #folder_dir = 'C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Documents/Progida Postdoc/Image data/ER LYS images from Amalie/TIFF files/20240314/TIFFs/L24Q_LysCh3' #Lysosomes in channel 3
    #folder_dir = 'C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Documents/Progida Postdoc/Image data/ER LYS images from Amalie/TIFF files/20240314/TIFFs' #Lysosomes in channel 1
    folder_dir = 'C:/Users/jonatmt/OneDrive - Universitetet i Oslo/Documents/Progida Postdoc/Image data/ER LYS images from Amalie/TIFF files/20240318/TIFFs' #Lysosomes in channel 1
    automated_analysis(folder_dir)


Processing image 1/39: control_ER-GFP_LysoTracker-Red_01_2024-03-18_Snap_16.04.22_CZC1467MB3.ims.tif
  Analyzing cell 1/3
  Analyzing cell 2/3
  Analyzing cell 3/3
Processing image 2/39: control_ER-GFP_LysoTracker-Red_02_2024-03-18_Snap_16.06.58_CZC1467MB3.ims.tif
  Analyzing cell 1/2
  Analyzing cell 2/2
Processing image 3/39: control_ER-GFP_LysoTracker-Red_03_2024-03-18_Snap_16.14.40_CZC1467MB3.ims.tif
  Analyzing cell 1/2
  Analyzing cell 2/2
Processing image 4/39: control_ER-GFP_LysoTracker-Red_04_2024-03-18_Snap_16.16.09_CZC1467MB3.ims.tif
  Analyzing cell 1/2
  Analyzing cell 2/2
Processing image 5/39: control_ER-GFP_LysoTracker-Red_05_2024-03-18_Snap_16.18.49_CZC1467MB3.ims.tif
  Analyzing cell 1/3
  Analyzing cell 2/3
  Analyzing cell 3/3
Processing image 6/39: control_ER-GFP_LysoTracker-Red_06_2024-03-18_Snap_16.20.50_CZC1467MB3.ims.tif
  Analyzing cell 1/2
  Analyzing cell 2/2
Processing image 7/39: control_ER-GFP_LysoTracker-Red_07_2024-03-18_Snap_16.23.57_CZC1467MB3.ims.tif